In [1]:
import json
import sqlite3
from sqlite3 import Error
from dateutil.parser import parse

In [2]:
def create_connection(db_file):
    conn = None
    
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)
    
    return conn

In [3]:
def define_db_table(conn):
    persons_table = """create table if not exists persons (
                                    id integer primary key ,
                                    name varchar(100) unique,
                                    role text,
                                    publications integer default 0,
                                    words integer default 0,
                                    first_seen date,
                                    last_seen date
                                );"""
    publications_table = """create table if not exists publications (
                                    id integer primary key ,
                                    pub_date date,
                                    nyt_id text,
                                    has_duplicated boolean default false,
                                    duplicate_amount integer default 0,
                                    material_id integer,
                                    news_desk_id integer,
                                    section_id integer,
                                    sub_section_id integer,
                                    foreign key (material_id) references type_of_material(id),
                                    foreign key (news_desk_id) references news_desks(id),
                                    foreign key (section_id) references sections(id),
                                    foreign key (sub_section_id) references sub_sections(id)
                                );"""
    collaborations_table = """create table if not exists collaborations (
                                    id integer primary key,
                                    rank integer not null,
                                    publication_id integer,
                                    person_id integer,
                                    foreign key (publication_id) references publications(id),
                                    foreign key (person_id) references persons(id)
                                );"""
    type_of_material_table ="""create table if not exists type_of_material (
                                    id integer primary key,
                                    name text not null unique
                                );"""
    news_desks_table ="""create table if not exists news_desks (
                                    id integer primary key,
                                    name text not null unique
                                );"""
    section_table ="""create table if not exists sections (
                                    id integer primary key,
                                    name text not null unique
                                );"""
    sub_section_table ="""create table if not exists sub_sections (
                                    id integer primary key,
                                    name text not null unique
                                );"""

    if conn is not None:
        try:
            print('create tables')
            c = conn.cursor()
            c.execute(type_of_material_table)
            c.execute(news_desks_table)
            c.execute(section_table)
            c.execute(sub_section_table)
            c.execute(persons_table)
            c.execute(publications_table)
            c.execute(collaborations_table)
        except Error as error:
            print("cannot create table", error)
    else:
        print("Error! cannot create the database connection.")

In [4]:
def db_execute(conn, query, entry):
    with conn:
        try:
            c = conn.cursor()
            c.execute(query, entry)
            conn.commit()
            return c.lastrowid
        except Error as e:
            pass

In [5]:
def find_one(conn, query, value):
    c = conn.cursor()
    c.execute(query, value)
    return c.fetchone()

In [6]:
def find_all(conn, query, value):
    c = conn.cursor()
    c.execute(query, value)
    return c.fetchall()

In [7]:
def get_fullname(_person):
    return f"{_person['firstname']} {_person['middlename']} {_person['lastname']}" if _person['middlename'] is not None else f"{_person['firstname']} {_person['lastname']}"

In [8]:
def static(db):
    with db:
        for year in range(2016, 2021):
            for month in range(1, 13):
                print(year, month)
                with open(f'../data/{year}_{month}.json') as json_file:
                    data = json.load(json_file)

                docs = data['response']['docs']

                for doc in docs:
                    # populate tables
                    if 'type_of_material' in doc:
                        db_execute(db, 'insert into type_of_material(name) values(?)', (doc['type_of_material'],))
                    if 'news_desk' in doc:
                        db_execute(db, 'insert into news_desks(name) values(?)', (doc['news_desk'],))
                    if 'section_name' in doc:
                        db_execute(db, 'insert into sections(name) values(?)', (doc['section_name'],))
                    if 'subsection_name' in doc:
                        db_execute(db, 'insert into sub_sections(name) values(?)', (doc['subsection_name'],))

                    # populate persons
                    try:
                        persons = doc['byline']['person']
                        if persons is None:
                            continue

                        publication_id = ''
                        material_id = ''
                        news_desk_id = ''
                        section_id = ''
                        sub_section_id = ''

                        if len(persons) > 0:
                            if 'type_of_material' in doc:
                                material_id = find_one(db, 'select * from type_of_material where name = ?', (doc['type_of_material'],))[0]
                            if 'news_desk' in doc:
                                news_desk_id = find_one(db, 'select * from news_desks where name = ?', (doc['news_desk'],))[0]
                            if 'section_name' in doc:
                                section_id = find_one(db, 'select * from sections where name = ?', (doc['section_name'],))[0]
                            if 'subsection_name' in doc:
                                sub_section_id = find_one(db, 'select * from sub_sections where name = ?', (doc['subsection_name'],))[0]

                            duplicates = find_all(db, 'select * from publications where nyt_id = ?', (doc['_id'],))
                            flag = len(duplicates) > 0
                            publication_id = db_execute(db, 'insert into publications(pub_date,nyt_id,has_duplicated,duplicate_amount,material_id,news_desk_id,section_id,sub_section_id) values(?,?,?,?,?,?,?,?)',
                                                (doc['pub_date'], doc['_id'], flag, len(duplicates), material_id, news_desk_id, section_id, sub_section_id))

                        for person in persons:
                            person_id = ''
                            name = get_fullname(person)
                            role = person['role']
                            words = doc['word_count']
                            person_entry = find_one(db, 'select * from persons where name = ?', (name,))

                            if person_entry is not None:
                                person_id = person_entry[0]
                                publications = person_entry[3] + 1
                                words = person_entry[4] + words
                                last_seen =  doc['pub_date'] if parse(person_entry[6]) < parse(doc['pub_date']) else person_entry[6]
                                db_execute(db,'update persons set publications = ? , words = ? , last_seen = ? where id = ?',
                                           (publications, words, last_seen, person_entry[0]))
                            else:
                                person_id = db_execute(db, 'insert into persons(name,role,publications,words,first_seen,last_seen) values(?,?,?,?,?,?)',
                                       (name,role,1,words,doc['pub_date'],doc['pub_date']))

                            db_execute(db, 'insert into collaborations(rank,publication_id,person_id) values(?,?,?)',
                                    (person['rank'], publication_id, person_id))

                    except Error as e:
                        print(e)
        print('end')

In [9]:
def main():
    db = create_connection(r"../teams.db")
    define_db_table(db)
    static(db)
    db.close()

In [10]:
main()

create tables
2016 1
2016 2
2016 3
2016 4
2016 5
2016 6
2016 7
2016 8
2016 9
2016 10
2016 11
2016 12
2017 1
2017 2
2017 3
2017 4
2017 5
2017 6
2017 7
2017 8
2017 9
2017 10
2017 11
2017 12
2018 1
2018 2
2018 3
2018 4
2018 5
2018 6
2018 7
2018 8
2018 9
2018 10
2018 11
2018 12
2019 1
2019 2
2019 3
2019 4
2019 5
2019 6
2019 7
2019 8
2019 9
2019 10
2019 11
2019 12
2020 1
2020 2
2020 3
2020 4
2020 5
2020 6
2020 7
2020 8
2020 9
2020 10
2020 11
2020 12


FileNotFoundError: [Errno 2] No such file or directory: '../data/2020_12.json'